In [ ]:
# @title Install
!apt-get install colmap ffmpeg

In [ ]:
# @title Upload video file.
# @markdown Select a video file (.mp4, .mov, etc.) from your disk. This will upload it to the local Colab working directory.
from google.colab import files

uploaded = files.upload()

Saving test.mp4 to test.mp4


In [ ]:
# @title Extract video frames
import sys
import os
import shutil
import cv2
from pathlib import Path

video = next(iter(uploaded.keys()))
video_name = os.path.basename(video).split('.')[0]
colmap_dir = Path('./custom')
rgb_dir = Path('./custom/images')
if os.path.exists(rgb_dir):
  shutil.rmtree(rgb_dir)
os.makedirs(rgb_dir)

frames = []
cap = cv2.VideoCapture(video)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

frames = frames[:150] # Avoid long-time training
i = 0
for frame in frames:
  cv2.imwrite(os.path.join(rgb_dir, f'{video_name}_{i:06d}.png'), frame)
  i += 1

print(f'Extracted {i} frames from {video}')
print(f'Frames are saved in {rgb_dir}')

Extracted 150 frames from test.mp4
Frames are saved in custom/images


### Camera registration with COLMAP.

In [ ]:
# @title Clone LLFF Util
%cd /content
!git clone https://github.com/Fyusion/LLFF

/content
Cloning into 'LLFF'...
remote: Enumerating objects: 774, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 774 (delta 7), reused 20 (delta 5), pack-reused 748
Receiving objects: 100% (774/774), 31.94 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (409/409), done.


In [ ]:
# @title Extract features.
%cd /content
import subprocess

share_intrinsics = True  # @param {type: 'boolean'}
assume_upright_cameras = True  # @param {type: 'boolean'}

# @markdown This sets the scale at which we will run COLMAP. A scale of 1 will be more accurate but will be slow.
colmap_db_path = colmap_dir / 'database.db'
colmap_out_path = colmap_dir / 'sparse'
colmap_out_path.mkdir(exist_ok=True, parents=True)

# @markdown Check this if you want to re-process SfM.
overwrite = True  # @param {type: 'boolean'}

if overwrite and colmap_db_path.exists():
  colmap_db_path.unlink()

!colmap feature_extractor \
--SiftExtraction.use_gpu 0 \
--database_path "{str(colmap_db_path)}" \
--image_path "{str(rgb_dir)}" \
--ImageReader.single_camera 1

/content

Feature extraction

Processed file [1/150]
  Name:            test_000000.png
  Dimensions:      512 x 512
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    614.40px
  Features:        2816
Processed file [2/150]
  Name:            test_000001.png
  Dimensions:      512 x 512
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    614.40px
  Features:        2783
Processed file [3/150]
  Name:            test_000002.png
  Dimensions:      512 x 512
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    614.40px
  Features:        2900
Processed file [4/150]
  Name:            test_000003.png
  Dimensions:      512 x 512
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    614.40px
  Features:        2978
Processed file [5/150]
  Name:            test_000004.png
  Dimensions:      512 x 512
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    614.40px
  Features:        2917
Processed file [6/150]
  Name:            test_000005.png
  Dimensions:      512 x 51

In [ ]:
# @title Match features.
# @markdown Match the SIFT features between images. Use `exhaustive` if you only have a few images and use `vocab_tree` if you have a lot of images.

match_method = 'vocab_tree'  # @param ["exhaustive", "vocab_tree"]

if match_method == 'exhaustive':
  !colmap exhaustive_matcher \
    --SiftMatching.use_gpu 0 \
    --database_path "{str(colmap_db_path)}"
else:
  # Use this if you have lots of frames.
  !wget https://demuc.de/colmap/vocab_tree_flickr100K_words32K.bin
  !colmap vocab_tree_matcher \
    --VocabTreeMatching.vocab_tree_path vocab_tree_flickr100K_words32K.bin \
    --SiftMatching.use_gpu 0 \
    --database_path "{str(colmap_db_path)}"

--2024-07-22 21:28:57--  https://demuc.de/colmap/vocab_tree_flickr100K_words32K.bin
Resolving demuc.de (demuc.de)... 78.46.140.213, 2a01:4f8:d0a:528e::2
Connecting to demuc.de (demuc.de)|78.46.140.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15229678 (15M) [application/octet-stream]
Saving to: ‘vocab_tree_flickr100K_words32K.bin.2’

vocab_tree_flickr10 100%[===================>]  14.52M  7.92MB/s    in 1.8s    

2024-07-22 21:29:00 (7.92 MB/s) - ‘vocab_tree_flickr100K_words32K.bin.2’ saved [15229678/15229678]


Vocabulary tree feature matching

Indexing image [1/150] in 0.329s
Indexing image [2/150] in 0.310s
Indexing image [3/150] in 0.333s
Indexing image [4/150] in 0.508s
Indexing image [5/150] in 0.583s
Indexing image [6/150] in 0.551s
Indexing image [7/150] in 0.439s
Indexing image [8/150] in 0.455s
Indexing image [9/150] in 0.446s
Indexing image [10/150] in 0.456s
Indexing image [11/150] in 0.282s
Indexing image [12/150] in 0.274s
Indexing image [1

In [ ]:
# @title Reconstruction.
# @markdown Run structure-from-motion to compute camera parameters.

refine_principal_point = True  #@param {type:"boolean"}
min_num_matches = 8 # @param {type: 'number'}
filter_max_reproj_error = 2  # @param {type: 'number'}
tri_complete_max_reproj_error = 2  # @param {type: 'number'}

!colmap mapper \
  --database_path $colmap_db_path \
  --image_path $rgb_dir \
  --output_path $colmap_out_path \
  --Mapper.ba_refine_principal_point true

串流輸出內容已截斷至最後 5000 行。

  => Merged observations: 84
  => Completed observations: 218
  => Filtered observations: 73
  => Changed observations: 0.022234

Bundle adjustment report
------------------------
    Residuals : 28048
   Parameters : 1187
   Iterations : 5
         Time : 0.121352 [s]
 Initial cost : 0.401481 [px]
   Final cost : 0.399471 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 365
  => Filtered observations: 0
  => Changed observations: 0.022693

Registering image #32 (79)

  => Image sees 2267 / 3860 points

Pose refinement report
----------------------
    Residuals : 4576
   Parameters : 6
   Iterations : 6
         Time : 0.025492 [s]
 Initial cost : 0.47326 [px]
   Final cost : 0.346081 [px]
  Termination : Convergence

  => Continued observations: 2248
  => Added observations: 934

Bundle adjustment report
------------------------
    Residuals : 29732
   Parameters : 1397
   Iterations : 13
         Time : 0.315441 [s]
 I

In [ ]:
# @title Run COLMAP
%cd /content
!python LLFF/imgs2poses.py $colmap_dir

/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxcb-icccm4 is already the newest version (0.4.1-1.1build2).
libxcb-image0 is already the newest version (0.4.0-2).
libxcb-keysyms1 is already the newest version (0.4.0-1build3).
libxcb-randr0 is already the newest version (1.14-3ubuntu3).
libxcb-xinerama0 is already the newest version (1.14-3ubuntu3).
libxcb-xkb1 is already the newest version (1.14-3ubuntu3).
libxcb1 is already the newest version (1.14-3ubuntu3).
libxcb-xrm0 is already the newest version (1.0-3).
libx11-xcb1 is already the newest version (2:1.7.5-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Don't need to run COLMAP
Post-colmap
Cameras 5
Images # 150
Points (16081, 3) Visibility (16081, 150)
Depth stats 0.008632815317383167 321.24384801473343 32.567930056923736
Done with imgs2poses


In [3]:
# @title Store camera poses
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
# !cp -r custom /content/gdrive/MyDrive/custom
!cp -r /content/gdrive/MyDrive/custom /content

### Run BoostMVSNeRFs

In [5]:
# @title Clone BoostMVSNeRFs Util
%cd /content
!git clone https://github.com/Su-Terry/BoostMVSNeRFs.git

/content
Cloning into 'BoostMVSNeRFs'...
remote: Enumerating objects: 1189, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1189 (delta 19), reused 83 (delta 13), pack-reused 1096
Receiving objects: 100% (1189/1189), 1.01 GiB | 43.81 MiB/s, done.
Resolving deltas: 100% (408/408), done.


In [6]:
%cd /content/BoostMVSNeRFs
!pip install -r requirements.txt

/content/BoostMVSNeRFs
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.3/833.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.9/695.9 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1

In [9]:
workspace = '/content'
dest_dir = os.path.join(workspace, 'trained_model/pretrain/custom')
!mkdir -p $dest_dir

import os
os.environ['workspace'] = '..'

file_id = '13OAVlcXgt7cGpFSTDvsy3SECH3KQEJ5N'
download_url = f'https://drive.google.com/uc?id={file_id}'
output_file = os.path.join(workspace, dest_dir, 'latest.pth')
!gdown -O $output_file $download_url

print(f"File downloaded to: {output_file}")
!ls $destination_dir


Downloading...
From: https://drive.google.com/uc?id=13OAVlcXgt7cGpFSTDvsy3SECH3KQEJ5N
To: /content/trained_model/pretrain/custom/latest.pth
100% 5.38M/5.38M [00:00<00:00, 129MB/s]
File downloaded to: /content/trained_model/pretrain/custom/latest.pth
BoostMVSNeRFs_DEMO.ipynb  data	lib	   requirements.txt  scripts	    train_net.py
configs			  docs	README.md  run.py	     trained_model  workspace


In [19]:
!bash scripts/custom.sh

Workspace:  ..
configs/exps/pretrain/enerf/dtu_pretrain.yaml
configs/exps/pretrain/enerf_ours/dtu_pretrain.yaml
configs/exps/evaluate/enerf_ours/base_eval.yaml
configs/exps/evaluate/enerf_ours/free_eval.yaml
configs/custom/custom.yaml
EXP NAME:  custom
View selection file not found. Preprocessing...
load model: ../trained_model/pretrain/custom/latest.pth
Preprocessing test set...
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 19/19 [12:25<00:00, 39.24s/it]
load model: ../trained_model/pretrain/custom/latest.pth
/usr/local/lib/python3.10/dist-packages/torch/util